In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1113095,2021-03-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1113096,2021-03-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1113097,2021-03-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1113098,2021-03-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
29979,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29981,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29983,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29985,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29987,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
687387,2021-03-23,Arkansas,Arkansas,5001,2027,35.00,5000,Arkansas,AR,Arkansas,State,3017804
687389,2021-03-24,Arkansas,Arkansas,5001,2031,35.00,5000,Arkansas,AR,Arkansas,State,3017804
687391,2021-03-25,Arkansas,Arkansas,5001,2068,36.00,5000,Arkansas,AR,Arkansas,State,3017804
687393,2021-03-26,Arkansas,Arkansas,5001,2071,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1113095,2021-03-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1113096,2021-03-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1113097,2021-03-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1113098,2021-03-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-27') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
431,2021-03-27,Snohomish,Washington,32426,558.00,53061,WA,County,822083,3944.37,67.88
860,2021-03-27,Cook,Illinois,493968,10183.00,17031,IL,County,5150233,9591.18,197.72
1288,2021-03-27,Orange,California,265653,4711.00,6059,CA,County,3175692,8365.20,148.35
1715,2021-03-27,Maricopa,Arizona,523088,9640.00,4013,AZ,County,4485414,11661.98,214.92
2142,2021-03-27,Los Angeles,California,1218359,23080.00,6037,CA,County,10039107,12136.13,229.90
...,...,...,...,...,...,...,...,...,...,...,...
1112559,2021-03-27,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1112725,2021-03-27,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1112860,2021-03-27,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1112991,2021-03-27,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
431,2021-03-27,Snohomish,Washington,32426,558.00,53061,WA,County,822083,3944.37,67.88,67.88,3944.37
860,2021-03-27,Cook,Illinois,493968,10183.00,17031,IL,County,5150233,9591.18,197.72,197.72,9591.18
1288,2021-03-27,Orange,California,265653,4711.00,6059,CA,County,3175692,8365.20,148.35,148.35,8365.20
1715,2021-03-27,Maricopa,Arizona,523088,9640.00,4013,AZ,County,4485414,11661.98,214.92,214.92,11661.98
2142,2021-03-27,Los Angeles,California,1218359,23080.00,6037,CA,County,10039107,12136.13,229.90,229.90,12136.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112559,2021-03-27,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1112725,2021-03-27,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1112860,2021-03-27,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1112991,2021-03-27,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
718696,2021-03-27,Hale,Alabama,2158,73.00,1065,AL,County,14651,14729.37,498.26,498.26,14729.37,1
782141,2021-03-27,Clarke,Alabama,3461,58.00,1025,AL,County,23622,14651.60,245.53,245.53,14651.60,2
562581,2021-03-27,Lowndes,Alabama,1358,53.00,1085,AL,County,9726,13962.57,544.93,544.93,13962.57,3
418776,2021-03-27,Franklin,Alabama,4184,82.00,1059,AL,County,31362,13340.99,261.46,261.46,13340.99,4
498923,2021-03-27,Etowah,Alabama,13632,342.00,1055,AL,County,102268,13329.68,334.42,334.42,13329.68,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685938,2021-03-27,Converse,Wyoming,982,17.00,56009,WY,County,13822,7104.62,122.99,122.99,7104.62,19
686303,2021-03-27,Sublette,Wyoming,693,7.00,56035,WY,County,9831,7049.13,71.20,71.20,7049.13,20
852343,2021-03-27,Lincoln,Wyoming,1328,12.00,56023,WY,County,19830,6696.92,60.51,60.51,6696.92,21
948279,2021-03-27,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
562581,2021-03-27,Lowndes,Alabama,1358,53.00,1085,AL,County,9726,13962.57,544.93,544.93,13962.57,3,1
718696,2021-03-27,Hale,Alabama,2158,73.00,1065,AL,County,14651,14729.37,498.26,498.26,14729.37,1,2
239564,2021-03-27,Walker,Alabama,7040,274.00,1127,AL,County,63521,11082.95,431.35,431.35,11082.95,28,3
606618,2021-03-27,Greene,Alabama,903,34.00,1063,AL,County,8111,11133.03,419.18,419.18,11133.03,26,4
561847,2021-03-27,Crenshaw,Alabama,1500,57.00,1041,AL,County,13772,10891.66,413.88,413.88,10891.66,30,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605153,2021-03-27,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
852343,2021-03-27,Lincoln,Wyoming,1328,12.00,56023,WY,County,19830,6696.92,60.51,60.51,6696.92,21,20
831162,2021-03-27,Uinta,Wyoming,2119,12.00,56041,WY,County,20226,10476.61,59.33,59.33,10476.61,4,21
238442,2021-03-27,Teton,Wyoming,3562,9.00,56039,WY,County,23464,15180.70,38.36,38.36,15180.70,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110477,2021-03-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1110478,2021-03-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1110479,2021-03-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1110480,2021-03-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
715716,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14729.37,2,1,1.00
715717,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14729.37,2,1,0.00
715718,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14729.37,2,1,0.00
715719,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14729.37,2,1,0.00
715720,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,498.26,14729.37,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866547,2021-03-23,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
866548,2021-03-24,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
866549,2021-03-25,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
866550,2021-03-26,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
560327,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13962.57,1,3,1.00,0.00
560328,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13962.57,1,3,0.00,0.00
560329,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13962.57,1,3,0.00,0.00
560330,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13962.57,1,3,0.00,0.00
560331,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13962.57,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558487,2021-03-23,Albany,Wyoming,3951,11.00,56001,WY,County,38880,10162.04,28.29,28.29,10187.76,23,7,4.00,0.00
558488,2021-03-24,Albany,Wyoming,3952,11.00,56001,WY,County,38880,10164.61,28.29,28.29,10187.76,23,7,1.00,0.00
558489,2021-03-25,Albany,Wyoming,3952,11.00,56001,WY,County,38880,10164.61,28.29,28.29,10187.76,23,7,0.00,0.00
558490,2021-03-26,Albany,Wyoming,3961,11.00,56001,WY,County,38880,10187.76,28.29,28.29,10187.76,23,7,9.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-27') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
285908,2021-03-27,Imperial,California,27374,711.00,6025,CA,County,181215,15105.81,392.35,392.35,15105.81,1,2,22.00,0.00
2142,2021-03-27,Los Angeles,California,1218359,23080.00,6037,CA,County,10039107,12136.13,229.90,229.90,12136.13,2,5,673.00,25.00
142498,2021-03-27,San Bernardino,California,290505,4288.00,6071,CA,County,2180085,13325.40,196.69,196.69,13325.40,3,4,190.00,66.00
500347,2021-03-27,Inyo,California,1369,35.00,6027,CA,County,18039,7589.11,194.02,194.02,7589.11,4,24,0.00,0.00
63785,2021-03-27,Stanislaus,California,58602,992.00,6099,CA,County,550660,10642.14,180.15,180.15,10642.14,5,9,0.00,0.00
79062,2021-03-27,Tulare,California,48960,813.00,6107,CA,County,466195,10502.04,174.39,174.39,10502.04,6,10,0.00,2.00
29592,2021-03-27,Riverside,California,293831,4248.00,6065,CA,County,2470546,11893.36,171.95,171.95,11893.36,7,6,0.00,0.00
50765,2021-03-27,San Joaquin,California,69448,1250.00,6077,CA,County,762148,9112.14,164.01,164.01,9112.14,8,19,0.00,0.00
28820,2021-03-27,Fresno,California,98731,1601.00,6019,CA,County,999101,9881.98,160.24,160.24,9881.98,9,13,129.00,8.00
606560,2021-03-27,Kings,California,22588,241.00,6031,CA,County,152940,14769.19,157.58,157.58,14769.19,10,3,10.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1056409,2021-03-27,Lassen,California,5649,1.00,6035,CA,County,30573,18477.09,3.27,3.27,18477.09,56,1,0.00,0.00
285908,2021-03-27,Imperial,California,27374,711.00,6025,CA,County,181215,15105.81,392.35,392.35,15105.81,1,2,22.00,0.00
606560,2021-03-27,Kings,California,22588,241.00,6031,CA,County,152940,14769.19,157.58,157.58,14769.19,10,3,10.00,0.00
142498,2021-03-27,San Bernardino,California,290505,4288.00,6071,CA,County,2180085,13325.40,196.69,196.69,13325.40,3,4,190.00,66.00
2142,2021-03-27,Los Angeles,California,1218359,23080.00,6037,CA,County,10039107,12136.13,229.90,229.90,12136.13,2,5,673.00,25.00
29592,2021-03-27,Riverside,California,293831,4248.00,6065,CA,County,2470546,11893.36,171.95,171.95,11893.36,7,6,0.00,0.00
179109,2021-03-27,Kern,California,105994,1211.00,6029,CA,County,900202,11774.47,134.53,134.53,11774.47,14,7,82.00,5.00
453501,2021-03-27,Merced,California,30419,434.00,6047,CA,County,277680,10954.70,156.30,156.30,10954.70,11,8,0.00,0.00
63785,2021-03-27,Stanislaus,California,58602,992.00,6099,CA,County,550660,10642.14,180.15,180.15,10642.14,5,9,0.00,0.00
79062,2021-03-27,Tulare,California,48960,813.00,6107,CA,County,466195,10502.04,174.39,174.39,10502.04,6,10,0.00,2.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
285908,2021-03-27,Imperial,California,27374,711.00,6025,CA,County,181215,15105.81,392.35,392.35,15105.81,1,2,22.00,0.00
2142,2021-03-27,Los Angeles,California,1218359,23080.00,6037,CA,County,10039107,12136.13,229.90,229.90,12136.13,2,5,673.00,25.00
142498,2021-03-27,San Bernardino,California,290505,4288.00,6071,CA,County,2180085,13325.40,196.69,196.69,13325.40,3,4,190.00,66.00
500347,2021-03-27,Inyo,California,1369,35.00,6027,CA,County,18039,7589.11,194.02,194.02,7589.11,4,24,0.00,0.00
63785,2021-03-27,Stanislaus,California,58602,992.00,6099,CA,County,550660,10642.14,180.15,180.15,10642.14,5,9,0.00,0.00
79062,2021-03-27,Tulare,California,48960,813.00,6107,CA,County,466195,10502.04,174.39,174.39,10502.04,6,10,0.00,2.00
29592,2021-03-27,Riverside,California,293831,4248.00,6065,CA,County,2470546,11893.36,171.95,171.95,11893.36,7,6,0.00,0.00
50765,2021-03-27,San Joaquin,California,69448,1250.00,6077,CA,County,762148,9112.14,164.01,164.01,9112.14,8,19,0.00,0.00
28820,2021-03-27,Fresno,California,98731,1601.00,6019,CA,County,999101,9881.98,160.24,160.24,9881.98,9,13,129.00,8.00
606560,2021-03-27,Kings,California,22588,241.00,6031,CA,County,152940,14769.19,157.58,157.58,14769.19,10,3,10.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15105.81,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15105.81,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15105.81,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15105.81,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15105.81,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3742,03/23/21,Lassen,5637,1.00,18437.84,3.27,3.27,18477.09,56,1,0.00,0.00
3743,03/24/21,Lassen,5637,1.00,18437.84,3.27,3.27,18477.09,56,1,0.00,0.00
3744,03/25/21,Lassen,5637,1.00,18437.84,3.27,3.27,18477.09,56,1,0.00,0.00
3745,03/26/21,Lassen,5649,1.00,18477.09,3.27,3.27,18477.09,56,1,12.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15105.81,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15105.81,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15105.81,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15105.81,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15105.81,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3825,03/23/21,Kings,22532,239.00,14732.57,156.27,157.58,14769.19,10,3,8.00,1.00
3826,03/24/21,Kings,22546,239.00,14741.73,156.27,157.58,14769.19,10,3,14.00,0.00
3827,03/25/21,Kings,22561,239.00,14751.54,156.27,157.58,14769.19,10,3,15.00,0.00
3828,03/26/21,Kings,22578,241.00,14762.65,157.58,157.58,14769.19,10,3,17.00,2.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)